In [1]:
!pip install sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

In [2]:
!pip install google-play-scraper

In [3]:
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
seed = 0
np.random.seed(seed)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score

In [4]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [5]:
import datetime as dt
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [6]:
from wordcloud import WordCloud

In [7]:
from google_play_scraper import reviews, Sort
import pandas as pd
import os

# Ganti dengan package name aplikasi target
app_id = "id.dana"

# Ambil 20000 review terbaru
result, _ = reviews(
    app_id,
    lang="id",
    country="id",
    sort=Sort.NEWEST,
    count=20000
)

# Convert ke DataFrame
df = pd.DataFrame(result)

# Simpan ke folder /content/data/
os.makedirs("/content/data", exist_ok=True)
df.to_csv("/content/data/ulasan_dana.csv", index=False)

print("✅ Scraping selesai, data tersimpan di /content/data/ulasan_dana.csv")
print("Kolom tersedia:", df.columns.tolist())
print(df.head())

'''
# Kalau mau download ke lokal
from google.colab import files
files.download("/content/data/ulasan_dana.csv")
'''


✅ Scraping selesai, data tersimpan di /content/data/ulasan_dana.csv
Kolom tersedia: ['reviewId', 'userName', 'userImage', 'content', 'score', 'thumbsUpCount', 'reviewCreatedVersion', 'at', 'replyContent', 'repliedAt', 'appVersion']
                               reviewId         userName  \
0  fff49d24-40df-4446-99f3-483f2c291035  Pengguna Google   
1  7436691b-eebd-4298-8ace-21cbcbe66be2  Pengguna Google   
2  35f7bf97-6a7d-4932-af9e-f44f2d56f711  Pengguna Google   
3  1643a4ce-6ca3-4421-909c-8b57670d7664  Pengguna Google   
4  bf0559f5-4246-46d2-8e8e-28e65dcbc80c  Pengguna Google   

                                           userImage  \
0  https://play-lh.googleusercontent.com/EGemoI2N...   
1  https://play-lh.googleusercontent.com/EGemoI2N...   
2  https://play-lh.googleusercontent.com/EGemoI2N...   
3  https://play-lh.googleusercontent.com/EGemoI2N...   
4  https://play-lh.googleusercontent.com/EGemoI2N...   

                                             content  score  thumbsUpC

'\n# Kalau mau download ke lokal\nfrom google.colab import files\nfiles.download("/content/data/ulasan_dana.csv")\n'

In [8]:
print(df.info())
print(df.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   reviewId              20000 non-null  object        
 1   userName              20000 non-null  object        
 2   userImage             20000 non-null  object        
 3   content               20000 non-null  object        
 4   score                 20000 non-null  int64         
 5   thumbsUpCount         20000 non-null  int64         
 6   reviewCreatedVersion  16320 non-null  object        
 7   at                    20000 non-null  datetime64[ns]
 8   replyContent          7844 non-null   object        
 9   repliedAt             7844 non-null   datetime64[ns]
 10  appVersion            16320 non-null  object        
dtypes: datetime64[ns](2), int64(2), object(7)
memory usage: 1.7+ MB
None
reviewId                    0
userName                    0
use

In [9]:
print(df.duplicated(subset=['content']).sum())

6572


In [10]:
# 1. Hapus baris dengan content kosong
df = df.dropna(subset=['content'])

# 2. Hapus duplikasi berdasarkan isi review
df = df.drop_duplicates(subset=['content'])

# 3. Isi NaN pada kolom appVersion & reviewCreatedVersion dengan "unknown"
df['appVersion'] = df['appVersion'].fillna("unknown")
df['reviewCreatedVersion'] = df['reviewCreatedVersion'].fillna("unknown")

# 4. Kolom replyContent & repliedAt mayoritas kosong → bisa dihapus jika tidak relevan
df = df.drop(columns=['replyContent',
                      'repliedAt',
                      'userImage',
                      'reviewCreatedVersion',
                      'appVersion',
                      'thumbsUpCount'
                      ])

# 5. Reset index setelah pembersihan
df = df.reset_index(drop=True)

# 6. Simpan hasil bersih
df.to_csv("/content/data/ulasan_dana_bersih.csv", index=False)

print("✅ Data sudah dibersihkan dan disimpan di /content/data/ulasan_dana_bersih.csv")
print("Jumlah baris setelah cleaning:", len(df))
print("Kolom tersedia:", df.columns.tolist())

✅ Data sudah dibersihkan dan disimpan di /content/data/ulasan_dana_bersih.csv
Jumlah baris setelah cleaning: 13428
Kolom tersedia: ['reviewId', 'userName', 'content', 'score', 'at']
